# English Premier League (EPL) Pythagorean Predictor

This project is my solution to the week 1 assignment of the course “Foundations of Sports analytics from the University of Michigan on coursera.

These are the steps you need to take to complete the assignment.:

a. Load the datafile (this contains 6 variables: the date, home team, away team, goals scored (FTHG), goals against (FTAG) andthe result (H- home win, D- draw, A – away win).  
b. Create a value for a home wins (win= 1, draw=0.5, loss= 0) and away wins and a count variable for each game (=1).  
c. Create a file for games played in 2017 (before date 20180000) and another one for games player in 2018 (after date 20180000)  
d. For the 2017 games, use .groupby to aggregate for each team the variables for count, home wins, away wins, goals for and goals against, first by the home team (one df), then by the away team (another df). Rename the variables to denote whether they are aggregates for home team or away team.
Then merge the home and away dataframes.  
e. Sum the values of home and away wins, games, goals for and goals against, then create the values for win percentage (wpc) and the Pythagorean expectation (pyth).   
f. Now repeat steps 4-6 for the 2018 games. Be sure to give different names for wpc and pyth in 2017 and 2018.  
g. Now merge 2017 and 2018 summary files.  
h. Now generate a correlation matrix for the wpc and pyth variables for 2017 and 2018  

In [1]:
# Load the packages

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
# Load the data. 
# EPL results for the 2017/18 season

EPL18 = pd.read_excel('Assignment Data/Week 1/EPL2017-18.xlsx')
print(EPL18.columns.tolist())
EPL18.head(10)

['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,20170811,Arsenal,Leicester,4,3,H
1,20170812,Brighton,Man City,0,2,A
2,20170812,Chelsea,Burnley,2,3,A
3,20170812,Crystal Palace,Huddersfield,0,3,A
4,20170812,Everton,Stoke,1,0,H
5,20170812,Southampton,Swansea,0,0,D
6,20170812,Watford,Liverpool,3,3,D
7,20170812,West Brom,Bournemouth,1,0,H
8,20170813,Man United,West Ham,4,0,H
9,20170813,Newcastle,Tottenham,0,2,A


In [19]:
# Create a value for a home wins (win= 1, draw=0.5, loss= 0) and away wins and a count variable for each game (=1).
EPL18['homewin'] = np.where(EPL18['FTR']== 'H',1, np.where(EPL18['FTR']=='D',0.5,0))
EPL18['awaywin'] = np.where(EPL18['FTR']== 'A',1, np.where(EPL18['FTR']=='D',0.5,0))
EPL18['count'] = 1
EPL18.head(15)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,homewin,awaywin,count
0,20170811,Arsenal,Leicester,4,3,H,1.0,0.0,1
1,20170812,Brighton,Man City,0,2,A,0.0,1.0,1
2,20170812,Chelsea,Burnley,2,3,A,0.0,1.0,1
3,20170812,Crystal Palace,Huddersfield,0,3,A,0.0,1.0,1
4,20170812,Everton,Stoke,1,0,H,1.0,0.0,1
5,20170812,Southampton,Swansea,0,0,D,0.5,0.5,1
6,20170812,Watford,Liverpool,3,3,D,0.5,0.5,1
7,20170812,West Brom,Bournemouth,1,0,H,1.0,0.0,1
8,20170813,Man United,West Ham,4,0,H,1.0,0.0,1
9,20170813,Newcastle,Tottenham,0,2,A,0.0,1.0,1


In [31]:
#For the 2017 games, use .groupby to aggregate for each team the variables for count, home wins, away wins, goals for and goals against, 
#first by the home team (one df), then by the away team (another df).
EPL2017 = EPL18[EPL18.Date < 20180000]
EPL17h = EPL2017.groupby(['HomeTeam'])['count','homewin','FTHG','FTAG'].sum().reset_index()
EPL17h = EPL17h.rename(columns={'HomeTeam':'team','count':'Ph','FTHG':'FTHGh','FTAG':'FTAGh'})
EPL17h

,team,Ph,homewin,FTHGh,FTAGh
0,Arsenal,10,8.5,25,10
1,Bournemouth,11,4.5,14,17
2,Brighton,10,5.5,10,12
3,Burnley,10,6.0,7,6
4,Chelsea,11,8.5,21,7
5,Crystal Palace,11,5.0,14,18
6,Everton,10,6.5,17,13
7,Huddersfield,11,6.0,10,12
8,Leicester,10,5.0,13,14
9,Liverpool,11,8.5,21,4


In [33]:
#Rename the variables to denote whether they are aggregates for home team or away team.
EPL17a = EPL2017.groupby(["AwayTeam"])['count','awaywin','FTHG','FTAG'].sum().reset_index()
EPL17a = EPL17a.rename(columns={'AwayTeam':'team','count':'Pa','FTHG':'FTHGa','FTAG':'FTAGa'})
EPL17a

,team,Pa,awaywin,FTHGa,FTAGa
0,Arsenal,11,5.0,16,13
1,Bournemouth,10,3.0,15,6
2,Brighton,11,3.0,13,5
3,Burnley,11,6.5,11,11
4,Chelsea,10,7.0,7,18
5,Crystal Palace,10,2.5,14,4
6,Everton,11,3.5,19,8
7,Huddersfield,10,3.0,20,8
8,Leicester,11,5.0,18,18
9,Liverpool,10,6.5,20,27


In [37]:
#Then merge the home and away dataframes.
EPL17 = pd.merge(EPL17h, EPL17a, on = 'team')
EPL17['Games'] = EPL17['Ph'] + EPL17['Pa']
EPL17['Wins'] = EPL17['homewin'] + EPL17['awaywin']
EPL17['GF'] = EPL17['FTHGh'] + EPL17['FTAGa']
EPL17['GA'] = EPL17['FTAGh'] + EPL17['FTHGa']
EPL17['wpc'] = EPL17['Wins']/EPL17['Games']
EPL17['pyth'] = EPL17['GF']**2/(EPL17['GF']**2 + EPL17['GA']**2)
EPL17

,team,Ph,homewin,FTHGh,FTAGh,Pa,awaywin,FTHGa,FTAGa,Games,Wins,GF,GA,wpc,pyth
0,Arsenal,10,8.5,25,10,11,5.0,16,13,21,13.5,38,26,0.642857,0.681132
1,Bournemouth,11,4.5,14,17,10,3.0,15,6,21,7.5,20,32,0.357143,0.280899
2,Brighton,10,5.5,10,12,11,3.0,13,5,21,8.5,15,25,0.404762,0.264706
3,Burnley,10,6.0,7,6,11,6.5,11,11,21,12.5,18,17,0.595238,0.528548
4,Chelsea,11,8.5,21,7,10,7.0,7,18,21,15.5,39,14,0.738095,0.885847
5,Crystal Palace,11,5.0,14,18,10,2.5,14,4,21,7.5,18,32,0.357143,0.240356
6,Everton,10,6.5,17,13,11,3.5,19,8,21,10.0,25,32,0.476190,0.379018
7,Huddersfield,11,6.0,10,12,10,3.0,20,8,21,9.0,18,32,0.428571,0.240356
8,Leicester,10,5.0,13,14,11,5.0,18,18,21,10.0,31,32,0.476190,0.484131
9,Liverpool,11,8.5,21,4,10,6.5,20,27,21,15.0,48,24,0.714286,0.800000


In [38]:
EPL17.describe()

,Ph,homewin,FTHGh,FTAGh,Pa,awaywin,FTHGa,FTAGa,Games,Wins,GF,GA,wpc,pyth
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.450000,6.000000,15.400000,12.350000,10.450000,4.450000,15.400000,12.350000,20.900000,10.450000,27.750000,27.750000,0.500000,0.461210
std,0.604805,1.926273,7.351334,5.050013,0.510418,2.108254,5.235506,6.706438,0.307794,3.831655,12.997469,8.638195,0.182915,0.270321
min,9.000000,3.000000,6.000000,4.000000,10.000000,2.500000,5.000000,4.000000,20.000000,6.000000,13.000000,12.000000,0.285714,0.141660
25%,10.000000,4.500000,10.000000,7.750000,10.000000,3.000000,12.750000,7.000000,21.000000,7.500000,18.000000,23.000000,0.357143,0.248367
50%,10.500000,5.250000,13.500000,13.000000,10.000000,3.250000,15.000000,10.500000,21.000000,9.000000,22.500000,30.000000,0.428571,0.343355
75%,11.000000,7.750000,21.000000,15.000000,11.000000,5.750000,18.250000,18.000000,21.000000,13.125000,38.250000,32.000000,0.644643,0.708793
max,11.000000,9.500000,36.000000,23.000000,11.000000,10.500000,27.000000,27.000000,21.000000,20.000000,61.000000,46.000000,0.952381,0.962743


## Step 4 (home team)

In [39]:
#Same process for 2018
EPL2018 = EPL18[EPL18.Date > 20180000]
EPL18h = EPL2018.groupby(['HomeTeam'])['count','homewin','FTHG','FTAG'].sum().reset_index()
EPL18h = EPL18h.rename(columns={'HomeTeam':'team','count':'Ph','FTHG':'FTHGh','FTAG':'FTAGh'})
EPL18h

,team,Ph,homewin,FTHGh,FTAGh
0,Arsenal,9,7.5,29,10
1,Bournemouth,8,5.0,12,13
2,Brighton,9,5.5,14,13
3,Burnley,9,3.5,9,11
4,Chelsea,8,4.5,9,9
5,Crystal Palace,8,4.5,15,9
6,Everton,9,5.5,11,9
7,Huddersfield,8,2.5,6,13
8,Leicester,9,5.0,12,8
9,Liverpool,8,7.0,24,6


In [40]:
EPL18a = EPL2018.groupby(['AwayTeam'])['count','awaywin','FTHG','FTAG'].sum().reset_index()
EPL18a = EPL18a.rename(columns={'AwayTeam':'team','count':'Pa','FTHG':'FTHGa','FTAG':'FTAGa'})
EPL18a

,team,Pa,awaywin,FTHGa,FTAGa
0,Arsenal,8,1.0,15,7
1,Bournemouth,9,4.0,16,13
2,Brighton,8,1.5,16,5
3,Burnley,8,4.0,11,9
4,Chelsea,9,4.5,15,14
5,Crystal Palace,9,4.5,14,12
6,Everton,8,2.5,17,8
7,Huddersfield,9,2.5,13,4
8,Leicester,8,2.5,20,13
9,Liverpool,9,5.0,8,12


In [43]:
Sum the values of home and away wins, games, goals for and goals against, 
EPL18 = pd.merge(EPL18h, EPL18a, on = 'team')
EPL18['Games'] = EPL18['Ph'] + EPL18['Pa']
EPL18['Wins'] = EPL18['homewin'] + EPL18['awaywin']
EPL18['GF'] = EPL18['FTHGh'] + EPL18['FTAGa']
EPL18['GA'] = EPL18['FTAGh'] + EPL18['FTHGa']
EPL18['wpc'] = EPL18['Wins']/EPL18['Games']
EPL18['pyth'] = EPL18['GF']**2/(EPL18['GF']**2 + EPL18['GA']**2)
EPL18

,team,Ph,homewin,FTHGh,FTAGh,Pa,awaywin,FTHGa,FTAGa,Games,Wins,GF,GA,wpc,pyth
0,Arsenal,9,7.5,29,10,8,1.0,15,7,17,8.5,36,25,0.500000,0.674649
1,Bournemouth,8,5.0,12,13,9,4.0,16,13,17,9.0,25,29,0.529412,0.426330
2,Brighton,9,5.5,14,13,8,1.5,16,5,17,7.0,19,29,0.411765,0.300333
3,Burnley,9,3.5,9,11,8,4.0,11,9,17,7.5,18,22,0.441176,0.400990
4,Chelsea,8,4.5,9,9,9,4.5,15,14,17,9.0,23,24,0.529412,0.478733
5,Crystal Palace,8,4.5,15,9,9,4.5,14,12,17,9.0,27,23,0.529412,0.579491
6,Everton,9,5.5,11,9,8,2.5,17,8,17,8.0,19,26,0.470588,0.348120
7,Huddersfield,8,2.5,6,13,9,2.5,13,4,17,5.0,10,26,0.294118,0.128866
8,Leicester,9,5.0,12,8,8,2.5,20,13,17,7.5,25,28,0.441176,0.443577
9,Liverpool,8,7.0,24,6,9,5.0,8,12,17,12.0,36,14,0.705882,0.868633


In [49]:
EPL18[['Games']].sum()

Games    342
dtype: int64

,Ph,homewin,FTHGh,FTAGh,Pa,awaywin,FTHGa,FTAGa,Games,Wins,GF,GA,wpc,pyth
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.00000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,8.550000,5.125000,13.700000,9.450000,8.550000,3.425000,13.70000,9.450000,17.100000,8.550000,23.150000,23.150000,0.499183,0.473341
std,0.604805,1.700426,6.165268,2.874113,0.510418,1.672337,3.70064,4.718106,0.307794,2.704285,9.143735,5.489464,0.154337,0.236570
min,8.000000,2.500000,6.000000,4.000000,8.000000,0.500000,8.00000,1.000000,17.000000,5.000000,10.000000,12.000000,0.294118,0.128866
25%,8.000000,4.250000,10.500000,8.000000,8.000000,2.500000,11.00000,5.000000,17.000000,6.500000,16.750000,20.750000,0.382353,0.294836
50%,8.500000,5.250000,12.000000,9.000000,9.000000,3.000000,14.50000,9.500000,17.000000,8.250000,21.500000,24.500000,0.485294,0.427632
75%,9.000000,6.250000,14.250000,11.250000,9.000000,4.500000,16.00000,12.250000,17.000000,9.000000,26.250000,27.250000,0.529412,0.604076
max,10.000000,7.500000,29.000000,14.000000,9.000000,6.500000,20.00000,20.000000,18.000000,14.000000,45.000000,30.000000,0.823529,0.900000


In [47]:
#Create the values for win percentage (wpc) and the Pythagorean expectation (pyth).
EPLpredictor = pd.merge(EPL17,EPL18, on='team')
EPLpredictor = EPLpredictor[['team', 'wpc_x', 'pyth_x', 'wpc_y', 'pyth_y']]
EPLpredictor

,team,wpc_x,pyth_x,wpc_y,pyth_y
0,Arsenal,0.642857,0.681132,0.500000,0.674649
1,Bournemouth,0.357143,0.280899,0.529412,0.426330
2,Brighton,0.404762,0.264706,0.411765,0.300333
3,Burnley,0.595238,0.528548,0.441176,0.400990
4,Chelsea,0.738095,0.885847,0.529412,0.478733
5,Crystal Palace,0.357143,0.240356,0.529412,0.579491
6,Everton,0.476190,0.379018,0.470588,0.348120
7,Huddersfield,0.428571,0.240356,0.294118,0.128866
8,Leicester,0.476190,0.484131,0.441176,0.443577
9,Liverpool,0.714286,0.800000,0.705882,0.868633


In [48]:
EPLpredictor.corr()

,wpc_x,pyth_x,wpc_y,pyth_y
wpc_x,1.000000,0.968204,0.756573,0.745832
pyth_x,0.968204,1.000000,0.795693,0.795331
wpc_y,0.756573,0.795693,1.000000,0.955986
pyth_y,0.745832,0.795331,0.955986,1.000000
